In [1]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

### minute level data

In [2]:
def process_dataset_min(df, symbol, datasets=None):
    df = df[df.symbol==symbol]

    df_backup = df.copy()
    df_backup = df_backup.reset_index()
    df_backup['date'] = df_backup['us_eastern_timestamp'].dt.date
    df_backup['hr'] = df_backup['us_eastern_timestamp'].dt.hour
    df_backup['min'] = df_backup['us_eastern_timestamp'].dt.minute

    if datasets is None:
        datasets = {}

    for dur in durations:
        new_dataset = df.resample(dur).agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum', 'trade_count': 'sum', 'market_open': 'max'})
        new_dataset = new_dataset.reset_index()
            
        new_dataset['date'] = new_dataset['us_eastern_timestamp'].dt.date
        new_dataset['hr'] = new_dataset['us_eastern_timestamp'].dt.hour
        new_dataset['min'] = new_dataset['us_eastern_timestamp'].dt.minute

        new_dataset = new_dataset.merge(df_backup[['date', 'hr', 'min']], on=['date', 'hr', 'min'])

        datasets[dur] = new_dataset.set_index('us_eastern_timestamp').drop(columns=['date', 'hr', 'min']).copy()
    return datasets

In [3]:
bucket_loc = 's3://sisyphus-general-bucket/AthenaInsights'
data_folder = 'latest_data'

In [4]:
# stock_data_day_level_name: stock_bars_day.parquet
# stock_bars_hour_level_name: stock_bars_hour.parquet
# stock_bars_minute_level_name: stock_bars_minute.parquet

In [5]:
import pandas as pd

def fill_missing_minutes(df, freq='1T'):
    """
    Fill missing minutes in financial time series data for each symbol, ensuring no data
    is generated outside of a day's trading hours.

    Parameters:
        df (pd.DataFrame): Dataframe with columns 'symbol', 'us_eastern_timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'.
        freq (str): Frequency for resampling, default is '1T' (one minute).

    Returns:
        pd.DataFrame: Dataframe with missing minutes filled within valid trading hours.
    """
    # Convert timestamp to datetime if not already
    df['us_eastern_timestamp'] = pd.to_datetime(df['us_eastern_timestamp'])

    # Set datetime as the index temporarily
    df.set_index('us_eastern_timestamp', inplace=True)

    # Function to resample each group while respecting daily bounds
    def resample_group(group):
        # Group by each day to respect daily boundaries
        daily_groups = []
        for name, day_group in group.groupby(group.index.date):
            min_time = day_group.index.min()
            max_time = day_group.index.max()

            # Resample within the day's min and max times
            resampled = day_group.resample(freq).ffill()
            resampled = resampled[(resampled.index >= min_time) & (resampled.index <= max_time)]

            # Fill missing data within the day
            resampled['open'].fillna(resampled['close'], inplace=True)
            resampled['high'].fillna(resampled['close'], inplace=True)
            resampled['low'].fillna(resampled['close'], inplace=True)
            resampled['volume'].fillna(0, inplace=True)
            resampled['trade_count'].fillna(0, inplace=True)
            resampled['vwap'].fillna(resampled['close'], inplace=True)

            daily_groups.append(resampled)

        # Combine all daily resampled groups
        return pd.concat(daily_groups)

    # Apply the resampling function to each symbol group
    filled_df = df.groupby('symbol').apply(resample_group)

    # Clean up the index
    filled_df.reset_index(level=0, drop=True, inplace=True)

    return filled_df.reset_index()

In [6]:
df = pd.read_parquet(f'{bucket_loc}/{data_folder}/parquet/stock_bars_minute.parquet')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['us_eastern_timestamp'] = df['timestamp'].dt.tz_convert('US/Eastern')
df['us_eastern_timestamp'] = df['us_eastern_timestamp'].dt.tz_localize(None)
df = df.drop(columns='timestamp')
df = df[df.symbol.isin(['SPY', 'QQQ'])]

assert df.open.isna().sum() == 0
assert df.high.isna().sum() == 0
assert df.low.isna().sum() == 0
assert df.close.isna().sum() == 0

# df['us_eastern_date'] = df.us_eastern_timestamp.dt.date
# df['market_open'] = df.us_eastern_timestamp.between('09:30:00', '16:00:00')
df = fill_missing_minutes(df)
df['market_open'] = (df.us_eastern_timestamp.dt.time>=pd.to_datetime('09:30:00').time()) & (df.us_eastern_timestamp.dt.time < pd.to_datetime('16:00:00').time())
df.set_index('us_eastern_timestamp', inplace=True)

In [7]:
df.head()

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02 04:00:00,QQQ,409.84,409.84,409.76,409.79,1541.0,26.0,409.799268,False
2024-01-02 04:01:00,QQQ,409.81,409.81,409.75,409.75,604.0,11.0,409.776912,False
2024-01-02 04:02:00,QQQ,409.71,409.71,409.68,409.68,1510.0,9.0,409.689310,False
2024-01-02 04:03:00,QQQ,409.71,409.71,409.66,409.66,2679.0,18.0,409.697663,False
2024-01-02 04:04:00,QQQ,409.63,409.63,409.61,409.61,343.0,6.0,409.623333,False


In [8]:
datasets = {}
symbol = df.symbol.unique() # 'SPY'
durations = ['2min', '3min', '5min', '10min', '15min', '20min', '25min', '30min']
for sym in symbol:
    datasets[sym] = {}
    datasets[sym]['1min'] = df[df.symbol==sym].copy()
    datasets[sym] = process_dataset_min(df, sym, datasets[sym])

In [9]:
datasets.keys(), datasets['SPY'].keys()

(dict_keys(['QQQ', 'SPY']),
 dict_keys(['1min', '2min', '3min', '5min', '10min', '15min', '20min', '25min', '30min']))

In [10]:
datasets['SPY']['1min']

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02 04:00:00,SPY,476.25,476.36,476.00,476.31,20460.0,84.0,476.301058,False
2024-01-02 04:01:00,SPY,476.34,476.34,476.29,476.29,6369.0,16.0,476.320154,False
2024-01-02 04:02:00,SPY,476.29,476.29,476.28,476.28,6152.0,10.0,476.280164,False
2024-01-02 04:03:00,SPY,476.27,476.27,476.27,476.27,369.0,10.0,476.270000,False
2024-01-02 04:04:00,SPY,476.27,476.27,476.27,476.27,369.0,10.0,476.270000,False
...,...,...,...,...,...,...,...,...,...
2024-11-19 19:55:00,SPY,591.53,591.53,591.50,591.52,1052.0,23.0,591.509655,False
2024-11-19 19:56:00,SPY,591.52,591.54,591.48,591.48,3823.0,47.0,591.498801,False
2024-11-19 19:57:00,SPY,591.46,591.46,591.41,591.42,3656.0,30.0,591.429199,False


In [11]:
datasets['SPY']['2min'].tail(182)

,open,high,low,close,volume,trade_count,market_open
us_eastern_timestamp,,,,,,,
2024-11-19 13:56:00,590.86,591.045,590.84,591.0300,79486.0,1067.0,1.0
2024-11-19 13:58:00,591.03,591.030,590.87,590.9000,84067.0,1173.0,1.0
2024-11-19 14:00:00,590.90,590.980,590.78,590.8400,466073.0,1906.0,1.0
2024-11-19 14:02:00,590.84,590.855,590.62,590.7103,98844.0,1286.0,1.0
2024-11-19 14:04:00,590.71,590.770,590.58,590.6421,129723.0,1312.0,1.0
...,...,...,...,...,...,...,...
2024-11-19 19:50:00,591.68,591.680,591.60,591.6099,2436.0,39.0,0.0
2024-11-19 19:52:00,591.58,591.660,591.55,591.5500,2156.0,39.0,0.0
2024-11-19 19:54:00,591.53,591.530,591.50,591.5200,2104.0,46.0,0.0


### hour level data

In [12]:
def process_dataset(df, symbol, datasets=None):
    df = df[df.symbol==symbol]
    if datasets is None:
        datasets = {}
    for dur in durations:
        new_dataset = df.resample(dur).agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
        datasets[dur] = new_dataset.copy()
    return datasets

In [13]:
# stock_data_day_level_name: stock_bars_day.parquet
# stock_bars_hour_level_name: stock_bars_hour.parquet
# stock_bars_minute_level_name: stock_bars_minute.parquet

In [14]:
df2 = pd.read_parquet(f'{bucket_loc}/{data_folder}/parquet/stock_bars_hour.parquet')
df2['timestamp'] = pd.to_datetime(df2['timestamp'])
df2['us_eastern_timestamp'] = df2['timestamp'].dt.tz_convert('US/Eastern')
df2['us_eastern_timestamp'] = df2['us_eastern_timestamp'].dt.tz_localize(None)
df2 = df2.drop(columns='timestamp')
df2 = df2[df2.symbol.isin(['SPY', 'QQQ'])]

# df2['us_eastern_date'] = df2.us_eastern_timestamp.dt.date
# df2['market_open'] = df2.us_eastern_timestamp.between('09:30:00', '16:00:00')
df2['market_open'] = (df2.us_eastern_timestamp.dt.time>=pd.to_datetime('09:30:00').time()) & (df2.us_eastern_timestamp.dt.time < pd.to_datetime('16:00:00').time())
df2.set_index('us_eastern_timestamp', inplace=True)

In [15]:
df2.head()

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02 04:00:00,QQQ,409.84,409.84,408.94,409.03,31737.0,355.0,409.288230,False
2024-01-02 05:00:00,QQQ,409.05,409.10,408.45,408.49,28039.0,234.0,408.816953,False
2024-01-02 06:00:00,QQQ,408.42,408.59,405.50,406.17,280578.0,1327.0,406.407732,False
2024-01-02 07:00:00,QQQ,406.19,406.25,404.83,404.99,308453.0,1862.0,405.626898,False
2024-01-02 08:00:00,QQQ,408.41,410.00,404.81,405.61,469161.0,4130.0,405.534781,False


In [16]:
symbol = df2.symbol.unique() # 'SPY'
durations = ['120min', '180min', '240min']
for sym in symbol:
    datasets[sym]['60min'] = df2[df2.symbol==sym].copy()
    datasets[sym] = process_dataset_min(df2, sym, datasets[sym])

In [17]:
datasets.keys(), datasets['SPY'].keys()

(dict_keys(['QQQ', 'SPY']),
 dict_keys(['1min', '2min', '3min', '5min', '10min', '15min', '20min', '25min', '30min', '60min', '120min', '180min', '240min']))

In [18]:
datasets['SPY']['60min']

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02 04:00:00,SPY,476.2500,476.3600,475.270,475.40,71308.0,349.0,475.991147,False
2024-01-02 05:00:00,SPY,475.3900,475.5100,475.100,475.10,43477.0,254.0,475.368634,False
2024-01-02 06:00:00,SPY,475.0100,475.0800,472.500,473.17,155953.0,951.0,473.294432,False
2024-01-02 07:00:00,SPY,473.0500,473.0600,471.700,471.79,422817.0,2270.0,472.470869,False
2024-01-02 08:00:00,SPY,473.0699,476.3500,471.640,472.03,568811.0,4569.0,472.096291,False
...,...,...,...,...,...,...,...,...,...
2024-11-19 15:00:00,SPY,589.6300,590.6000,589.365,590.29,9525290.0,66503.0,590.016005,True
2024-11-19 16:00:00,SPY,590.2800,590.8000,590.200,590.24,11162473.0,4219.0,590.378012,False
2024-11-19 17:00:00,SPY,590.1500,590.2199,590.000,590.12,315825.0,743.0,590.056293,False


In [19]:
datasets['SPY']['120min']

,open,high,low,close,volume,trade_count,market_open
us_eastern_timestamp,,,,,,,
2024-01-02 04:00:00,476.2500,476.360,475.100,475.1000,114785.0,603.0,0.0
2024-01-02 06:00:00,475.0100,475.080,471.700,471.7900,578770.0,3221.0,0.0
2024-01-02 08:00:00,473.0699,476.350,471.360,472.2700,13102075.0,102585.0,0.0
2024-01-02 10:00:00,472.2750,473.290,471.300,473.1850,18398010.0,176178.0,1.0
2024-01-02 12:00:00,473.1900,473.670,472.000,472.1499,17574781.0,116247.0,1.0
...,...,...,...,...,...,...,...
2024-11-19 10:00:00,585.2500,589.240,584.645,588.8290,8838561.0,119448.0,1.0
2024-11-19 12:00:00,588.8300,591.045,588.030,590.9000,7645645.0,72916.0,1.0
2024-11-19 14:00:00,590.9000,590.980,589.090,590.2900,15381051.0,114348.0,1.0


### day level data

In [20]:
# stock_data_day_level_name: stock_bars_day.parquet
# stock_bars_hour_level_name: stock_bars_hour.parquet
# stock_bars_minute_level_name: stock_bars_minute.parquet

In [21]:
df3 = pd.read_parquet(f'{bucket_loc}/{data_folder}/parquet/stock_bars_day.parquet')
df3['timestamp'] = pd.to_datetime(df3['timestamp'])
df3['us_eastern_timestamp'] = df3['timestamp'].dt.tz_convert('US/Eastern')
df3['us_eastern_timestamp'] = df3['us_eastern_timestamp'].dt.tz_localize(None)
df3 = df3.drop(columns='timestamp')
df3 = df3[df3.symbol.isin(['SPY', 'QQQ'])]

# df3['us_eastern_date'] = df3.us_eastern_timestamp.dt.date
# df3['market_open'] = df3.us_eastern_timestamp.between('09:30:00', '16:00:00')
df3['market_open'] = (df3.us_eastern_timestamp.dt.time>=pd.to_datetime('09:30:00').time()) & (df3.us_eastern_timestamp.dt.time < pd.to_datetime('16:00:00').time())
df3.set_index('us_eastern_timestamp', inplace=True)

In [22]:
df3.head()

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02,QQQ,405.84,406.09,400.2400,402.59,58073141.0,498841.0,402.642947,False
2024-01-03,QQQ,399.93,401.00,397.8900,398.33,47037517.0,421720.0,399.376503,False
2024-01-04,QQQ,396.44,399.59,396.0600,396.28,39471644.0,341643.0,397.541973,False
2024-01-05,QQQ,396.45,399.56,395.3400,396.75,44923110.0,405464.0,397.442473,False
2024-01-08,QQQ,397.99,405.24,397.8399,404.95,42543338.0,362296.0,402.636103,False


In [23]:
symbol = df3.symbol.unique() # 'SPY'
durations = ['2D', '3D', '5D', '10D', '15D', '20D', '30D', '50D', '100D', '150D', '200D']
for sym in symbol:
    datasets[sym]['1D'] = df3[df3.symbol==sym].copy()
    datasets[sym] = process_dataset(df3, sym, datasets[sym])

In [24]:
datasets[sym]['1D']

,symbol,open,high,low,close,volume,trade_count,vwap,market_open
us_eastern_timestamp,,,,,,,,,
2024-01-02,SPY,472.16,473.670,470.49,472.65,123007793.0,657585.0,472.044088,False
2024-01-03,SPY,470.43,471.190,468.17,468.79,103595966.0,656572.0,469.620528,False
2024-01-04,SPY,468.30,470.960,467.05,467.28,84232169.0,536471.0,468.783715,False
2024-01-05,SPY,467.49,470.440,466.43,467.92,86118913.0,562579.0,468.323224,False
2024-01-08,SPY,468.43,474.750,468.30,474.60,74879074.0,523806.0,472.074423,False
...,...,...,...,...,...,...,...,...,...
2024-11-13,SPY,597.37,599.230,594.96,597.19,47388640.0,399844.0,597.564075,False
2024-11-14,SPY,597.32,597.810,592.65,593.35,38904109.0,373549.0,594.785489,False
2024-11-15,SPY,589.72,590.200,583.86,585.75,75942463.0,536923.0,586.320940,False


In [25]:
datasets[sym]['2D']

,open,high,low,close,volume
us_eastern_timestamp,,,,,
2024-01-02,472.16,473.670,468.17,468.79,226603759.0
2024-01-04,468.30,470.960,466.43,467.92,170351082.0
2024-01-06,NaN,NaN,NaN,NaN,0.0
2024-01-08,468.43,474.930,468.30,473.88,140810513.0
2024-01-10,474.16,478.120,472.26,476.35,145251463.0
...,...,...,...,...,...
2024-11-11,599.81,600.170,594.37,596.90,80592901.0
2024-11-13,597.37,599.230,592.65,593.35,86292749.0
2024-11-15,589.72,590.200,583.86,585.75,75942463.0


### writing out to s3

In [26]:
all_durations = []
for sym in datasets.keys():
    for dur in datasets[sym].keys():
        all_durations.append(dur)
all_durations = set(all_durations)
print(all_durations)

{'3min', '15min', '30min', '10min', '15D', '150D', '2min', '120min', '1min', '3D', '2D', '5min', '20min', '20D', '240min', '25min', '100D', '200D', '5D', '60min', '1D', '30D', '50D', '10D', '180min'}


In [27]:
for dur in all_durations:
    dur_df = pd.DataFrame()
    for sym in datasets.keys():
        if dur in datasets[sym].keys():
            dur_df = pd.concat([dur_df, datasets[sym][dur].assign(symbol=sym)])
    dur_df.to_parquet(f'{bucket_loc}/{data_folder}/data_prep/stock_bars_{dur}.parquet')

### testing

In [28]:
# date_f = '2024-11-04'

In [29]:
# x = df[df.symbol=='SPY'].reset_index()
# x[(x.timestamp>=pd.to_datetime(f'2024-11-04 00:00:00+00:00'))&(x.timestamp<pd.to_datetime(f'2024-11-04 23:00:000+00:00'))].trade_count.sum()
# x[(x.us_eastern_timestamp>=pd.to_datetime(f'2024-11-04 00:00:00'))&(x.us_eastern_timestamp<pd.to_datetime(f'2024-11-04 23:00:00'))]#.trade_count.sum()

In [30]:
# y = df2[df2.symbol=='SPY']
# y[y.us_eastern_date==pd.to_datetime(date_f)]

In [31]:
# z = df3[df3.symbol=='SPY']
# z[z.us_eastern_date==pd.to_datetime(date_f)]